In [27]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn

In [28]:
df_original = pd.read_csv("results/hochuli/count.csv")
df_double = pd.read_csv("results/hochuli_double/count.csv")
df_deep = pd.read_csv("results/hochuli_deep/count.csv")
df_mobilenet = pd.read_csv("results/mobilenet/count.csv")

In [29]:
# fazendo a média por execução (cada época aparece num_execucoes)
df_original = df_original.groupby("epoch").mean()
df_double = df_double.groupby("epoch").mean()
df_deep = df_deep.groupby("epoch").mean()
df_mobilenet = df_mobilenet.groupby("epoch").mean()

In [30]:
df_deep.head(5)

,execution,train_loss,val_loss,val_acc,time,zero_relu_call,total_relu_call
epoch,,,,,,,
-1,3.0,NaN,0.333169,0.882420,1.729141,0.0,0.000000e+00
1,3.0,2.303282,2.302570,0.098200,19.601018,285.0,1.056384e+09
2,3.0,2.302126,2.301574,0.111200,19.168741,309.8,1.056384e+09
3,3.0,2.217693,1.982424,0.228178,19.218155,308.8,1.056384e+09
4,3.0,1.717483,1.644966,0.359289,19.185289,372.8,1.056384e+09


In [31]:
# variáveis de interesse
dados = {"original": df_original, "double": df_double, "deep": df_deep, "mobilenet": df_mobilenet}
resultados = {}
for nome, df in dados.items():
    relu0 = round(df["zero_relu_call"].sum())
    relux = round(df["total_relu_call"].sum())
    prop = relu0 / relux
    resultados[nome] = [relu0, relux, prop]

In [32]:
from model_utils import Hochuli, HochuliDeep, HochuliDoubleKernels, create_mobile_net
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

resultados["original"].append(count_parameters(Hochuli([3, 28, 28], 10, nn.ReLU())))
resultados["double"].append(count_parameters(HochuliDoubleKernels([3, 28, 28], 10, nn.ReLU())))
resultados["deep"].append(count_parameters(HochuliDeep([3, 28, 28], 10, nn.ReLU())))
resultados["mobilenet"].append(count_parameters(create_mobile_net(nn.ReLU(), nn.Hardswish(), nn.Hardsigmoid(), 10, pre_trained=True, freeze=False)))

In [33]:
linhas = ["ReLU'(0)", "ReLU'(x)", "ReLU'(0)/ReLU'(x)", "Parâmetros"]
pd.DataFrame(data=resultados, index=linhas)

,original,double,deep,mobilenet
ReLU'(0),3.971240e+05,7.029090e+05,9.918000e+03,2.580000e+03
ReLU'(x),5.281920e+10,1.055040e+11,5.281920e+10,5.779200e+10
ReLU'(0)/ReLU'(x),7.518554e-06,6.662392e-06,1.877726e-07,4.464286e-08
Parâmetros,7.341800e+04,2.567140e+05,1.104420e+05,1.528106e+06


In [34]:
resultados["original"][2] / resultados["deep"][2]

40.04073401895543

In [35]:
resultados["original"][2] / resultados["mobilenet"][2]

168.4156064461408